In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 색온도 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('./../swr_area_test.csv', delimiter=',', dtype='float')

x_data = import_data[:, :2]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
test_x = x_data[86:, :2]
x_data = x_data[:86, :2]

y_data = []
test_y = []

for i in range(86):
    temp = []
    temp.append(import_data[i][2])
    y_data.append(temp)

raw_y = y_data
    
for i in range(86, len(import_data)):
    temp = []
    temp.append(import_data[i][2])
    test_y.append(temp)
    
# raw_y = y_data

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 2])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

#기존 코드
# W = tf.Variable(tf.random_normal([11, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[2, nb_classes], initializer=xavier_init(2, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

#  dropout 추가 코드
# dropout_rate = tf.placeholder("float")

# W1 = tf.get_variable("W1", shape=[11, 30], initializer=xavier_init(11, 30))
# W2 = tf.get_variable("W2", shape=[30, 30], initializer=xavier_init(30, 30))
# W3 = tf.get_variable("W3", shape=[30, nb_classes], initializer=xavier_init(30, nb_classes))

# B1 = tf.Variable(tf.random_normal([30]))
# B2 = tf.Variable(tf.random_normal([30]))
# B3 = tf.Variable(tf.random_normal([5]))

# _L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), B1))
# L1 = tf.nn.dropout(_L1, dropout_rate)
# _L2 = tf.nn.relu(tf.add(tf.matmul(_L1, W2), B2))
# L2 = tf.nn.dropout(_L2, dropout_rate)

# H = tf.add(tf.matmul(_L2, W3), B3)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(150001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})

        if step % 10000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
        if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_x) * 100))

0 1.3006974
10000 1.2184391
20000 1.18856
30000 1.1641331
40000 1.1429254
50000 1.124126
60000 1.1072816
70000 1.0920728
80000 1.0782546
90000 1.0656233
100000 1.0540204
110000 1.0433071
120000 1.0333685
130000 1.0241107
140000 1.015452
150000 1.0073327
-----------------------------
train_data =  86 test_data =  37
[4] [1.0]
[4] [2.0]
[4] [3.0]
[3] [0.0]
[4] [3.0]
[3] [0.0]
[4] [1.0]
[4] [4.0]
[4] [2.0]
[4] [2.0]
[4] [2.0]
[4] [4.0]
[4] [4.0]
[4] [3.0]
[4] [3.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [2.0]
[4] [3.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [4.0]
[4] [3.0]
[4] [4.0]
[4] [3.0]
[4] [3.0]
[4] [4.0]
[4] [4.0]
accuracy =  54.054054054054056
